# Agenda: Pivot tables in Pandas

1. Grouping and aggregation
2. What are pivot tables?
3. Simple pivoting examples
4. Changing the aggregation function
5. Margins
6. Multiple aggregation functions
7. Multiple values

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# I'm going to create a simple data set -- sales figures for three products (A, B, and C) in different years and months

np.random.seed(0)   # reset the random-number generator
df = DataFrame(np.random.randint(0, 100, [36, 3]),
               columns=list('ABC'))

df['year'] = [2022] * 12 + [2023] * 12 + [2024] * 12
df['month'] = 'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split() * 3
df

In [5]:
# I want to get the mean sales figures for product A

df['A'].mean()

np.float64(50.97222222222222)

In [7]:
# I want to know the mean sales for product A, but only in 2022

(
    df
    .loc[df['year'] == 2022,   # row selector
         'A']  # column selector
    .mean()
)

np.float64(67.91666666666667)

In [8]:
# I want to know the mean sales for product A, but only in 2023

(
    df
    .loc[df['year'] == 2023,   # row selector
         'A']  # column selector
    .mean()
)

np.float64(32.333333333333336)

In [9]:
# I want to know the mean sales for product A, but only in 2024

(
    df
    .loc[df['year'] == 2024,   # row selector
         'A']  # column selector
    .mean()
)

np.float64(52.666666666666664)

# DRY -- don't repeat yourself

There must be a better way to do this!

- `groupby` -- is basically running the three above queries. To run a `groupby`, we need:

- one column which is *categorical*, meaning that it repeats throughout the data frame
- one column which is *numeric*
- an aggregation method

We're going to run an aggregation method on the numeric column for each of the categorical's unique values



In [10]:
# this means:
# for every distinct value of "year"
# calculate the mean 
# on column A

df.groupby('year')['A'].mean()

year
2022    67.916667
2023    32.333333
2024    52.666667
Name: A, dtype: float64

In [11]:
# we can do the same thing by month!

df.groupby('month')['A'].mean()

month
Apr    67.666667
Aug    63.000000
Dec    83.666667
Feb    27.000000
Jan    39.666667
Jul    41.333333
Jun    41.000000
Mar    67.000000
May    39.000000
Nov    29.666667
Oct    56.666667
Sep    56.000000
Name: A, dtype: float64

In [12]:
df.groupby('month', sort=False)['A'].mean()

month
Jan    39.666667
Feb    27.000000
Mar    67.000000
Apr    67.666667
May    39.000000
Jun    41.000000
Jul    41.333333
Aug    63.000000
Sep    56.000000
Oct    56.666667
Nov    29.666667
Dec    83.666667
Name: A, dtype: float64

In [13]:
# can I perform a groupby on more than one categorical?
# after all, I'd like to know the sales per year + month

# a good rule of thumb in Pandas is that wherever you can use one column (string), you can use more than one (as a list)

df.groupby(['year', 'month'], sort=False)['A'].mean()

year  month
2022  Jan      44.0
      Feb      67.0
      Mar      83.0
      Apr      87.0
      May      88.0
      Jun      65.0
      Jul      46.0
      Aug      37.0
      Sep      72.0
      Oct      80.0
      Nov      47.0
      Dec      99.0
2023  Jan      29.0
      Feb      14.0
      Mar      65.0
      Apr      32.0
      May      23.0
      Jun      55.0
      Jul       0.0
      Aug      53.0
      Sep      17.0
      Oct      42.0
      Nov       1.0
      Dec      57.0
2024  Jan      46.0
      Feb       0.0
      Mar      53.0
      Apr      84.0
      May       6.0
      Jun       3.0
      Jul      78.0
      Aug      99.0
      Sep      79.0
      Oct      48.0
      Nov      41.0
      Dec      95.0
Name: A, dtype: float64

# What is a pivot table?

It's the same thing as the 2D grouping operation that we just did! The only difference is that our two-level index is separated, such that one part remains on the rows (as the index) and the other part becomes the columns, turning it into a table.

A pivot table is just another way of doing/expressing a 2D groupby operation. It makes certain types of data and values much clearer. Moreover, the fact that we have a data frame produced by the operation means that we can then run all of our favorite data-frame operations.

# Let's recreate the above as a pivot table

I'll invoke the `pivot_table` method on my data frame. There is a `pivot` method, you probably don't want to use it.

- `index` -- we pass the name of the categorical that'll be used to label the rows
- `columns` -- we pass the name of the categorical that'll be used to label the columns
- `values` -- we name the column whose values will be computed
- `aggfunc` -- we name the aggregation method we want to use (as a string)

In [15]:
df.pivot_table(index='month',
               columns='year',
               values='A',
               aggfunc='mean', 
              sort=False)

year,2022,2023,2024
month,,,
Jan,44.0,29.0,46.0
Feb,67.0,14.0,0.0
Mar,83.0,65.0,53.0
Apr,87.0,32.0,84.0
May,88.0,23.0,6.0
Jun,65.0,55.0,3.0
Jul,46.0,0.0,78.0
Aug,37.0,53.0,99.0
Sep,72.0,17.0,79.0


In [17]:
# if you have our earlier groupby result, you can actually turn it into this pivot table 
# with the "unstack" method, which turns a part of the multi-index into columns

df.groupby(['year', 'month'])['A'].mean().unstack('year')

year,2022,2023,2024
month,,,
Apr,87.0,32.0,84.0
Aug,37.0,53.0,99.0
Dec,99.0,57.0,95.0
Feb,67.0,14.0,0.0
Jan,44.0,29.0,46.0
Jul,46.0,0.0,78.0
Jun,65.0,55.0,3.0
Mar,83.0,65.0,53.0
May,88.0,23.0,6.0


We'll use the taxi file (`taxi.csv`) from the zipfile at https://files.lerner.co.il/data-science-exercise-files.zip .

# Exercise: Taxi info

1. This file contains 10,000 taxi rides from NYC in about 2015. The columns we care about for this exercise are `payment_type`, `passenger_count`, and `total_amount`.
2. Find, for every combination of `payment_type` and `passenger_count`, the mean `total_amount`. In other words, given a way to pay and a number of passengers, which led to the highest payment for the taxi ride?

In [22]:
df = pd.read_csv('taxi.csv',
                usecols=['payment_type', 'passenger_count', 'total_amount', 'trip_distance'])

In [23]:
df

,passenger_count,trip_distance,payment_type,total_amount
0,1,1.63,2,17.80
1,1,0.46,1,8.30
2,1,0.87,1,11.00
3,1,2.13,1,17.16
4,1,1.40,2,10.30
...,...,...,...,...
9994,1,2.70,2,12.30
9995,1,4.50,1,20.30
9996,1,5.59,2,22.30
9997,6,1.54,2,7.80


In [24]:
# index: payment_type (categorical)
# columns: passenger_count (categorical)
# values: total_amount
# aggfunc: mean

df.pivot_table(index='payment_type',
               columns='passenger_count',
               values='total_amount',
               aggfunc='mean')

passenger_count,0,1,2,3,4,5,6
payment_type,,,,,,,
1,25.57,19.883125,20.934013,18.779292,22.454773,19.63475,18.980288
2,NaN,13.512674,14.892018,16.988436,15.536596,13.33370,15.361491
3,NaN,12.020645,42.480000,NaN,NaN,NaN,NaN
4,NaN,7.780000,NaN,20.800000,NaN,NaN,NaN


In [25]:
# what if I use trip_distance as my index column?

df.pivot_table(index='trip_distance',
               columns='passenger_count',
               values='total_amount',
               aggfunc='mean')

passenger_count,0,1,2,3,4,5,6
trip_distance,,,,,,,
0.00,NaN,31.702292,44.442857,46.98,25.685,9.05,3.30
0.01,NaN,NaN,NaN,52.80,NaN,NaN,NaN
0.02,NaN,3.800000,NaN,NaN,NaN,NaN,83.12
0.03,NaN,3.960000,NaN,NaN,NaN,NaN,NaN
0.04,NaN,70.010000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
34.84,NaN,137.590000,NaN,NaN,NaN,NaN,NaN
35.51,NaN,135.130000,NaN,NaN,NaN,NaN,NaN
37.20,NaN,210.140000,NaN,NaN,NaN,NaN,NaN


In [26]:
import seaborn as sns
tips = sns.load_dataset('tips')

In [27]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [29]:
# let's find the mean total_bill for smokers vs. non-smokers

tips.groupby('smoker', observed=True)['total_bill'].mean()

smoker
Yes    20.756344
No     19.188278
Name: total_bill, dtype: float64

In [30]:
# let's find the mean total_bill for different meals

tips.groupby('time', observed=True)['total_bill'].mean()

time
Lunch     17.168676
Dinner    20.797159
Name: total_bill, dtype: float64

In [32]:
# if I want to compare smokers and time in a pivot table, how do I do that?

# index = smoker
# columns = time
# values = total_bill
# aggfunc = mean

tips.pivot_table(index='smoker',
                 columns='time',
                 values='total_bill',
                 aggfunc='mean',
                observed=True)

time,Lunch,Dinner
smoker,,
Yes,17.399130,21.859429
No,17.050889,20.095660


In [36]:
# we can get the same thing via a 2D groupby and unstack

tips.groupby(['smoker', 'time'], observed=True)['total_bill'].mean().unstack('time')

time,Lunch,Dinner
smoker,,
Yes,17.399130,21.859429
No,17.050889,20.095660


# Exercise: Titanic pivots!

The file `titanic3.csv` has information on every passenger on the Titanic.

- What was the mean age of people on the Titanic, if we pivot on `sex` vs. `survived`?
- What was the mean fare that people paid, looking at `class` vs. `sex`?
- What was the mean fare people paid, by `class` vs. `home.dest`?

In [37]:
!head titanic3.csv

,,,,,,,,,,,,,n, Mr. Leo",male,29,0,0,315082,7.8750,,S,,,0,,C,,304,4.4542,,C,,,2,,C,C,, OH",,S,,,,N"S,15,,NY" Falls, NY"t, MI"A"rk, NY"tle, WA"in" PA"

In [38]:
df = pd.read_csv('titanic3.csv')

In [39]:
# What was the mean age of people on the Titanic, if we pivot on `sex` vs. `survived`?

df.pivot_table(index='sex',
               columns='survived',
               values='age',
               aggfunc='mean')

survived,0.0,1.0
sex,,
female,25.255208,29.815354
male,31.516412,26.977778


In [40]:
# What was the mean fare that people paid, looking at `class` vs. `sex`?

df.pivot_table(index='sex',
               columns='pclass',
               values='fare',
               aggfunc='mean')


pclass,1.0,2.0,3.0
sex,,,
female,109.412385,23.234827,15.324250
male,69.888385,19.904946,12.415462


In [41]:
# What was the mean fare people paid, by `class` vs. `home.dest`?

df.pivot_table(index='home.dest',
               columns='pclass',
               values='fare',
               aggfunc='mean')

pclass,1.0,2.0,3.0
home.dest,,,
"?Havana, Cuba",27.7208,NaN,NaN
"Aberdeen / Portland, OR",NaN,15.75,NaN
"Albany, NY",28.5375,NaN,NaN
"Altdorf, Switzerland",NaN,NaN,17.8
"Amenia, ND",61.1750,NaN,NaN
...,...,...,...
"Worcester, England",NaN,26.00,NaN
"Worcester, MA",52.0000,13.00,NaN
"Yoevil, England / Cottage Grove, OR",NaN,13.50,NaN


In [42]:
# where was the first-class ticket > 100? 

(
    df.pivot_table(index='home.dest',
                   columns='pclass',
                   values='fare',
                   aggfunc='mean')
    .loc[lambda df_: df_[1.0] > 100]  # only keep rows where the value in the 1.0 column is > 100
)

pclass,1.0,2.0,3.0
home.dest,,,
"Austria-Hungary / Germantown, Philadelphia, PA",512.329200,NaN,NaN
"Bryn Mawr, PA",120.000000,NaN,NaN
"Cooperstown, NY",262.375000,NaN,NaN
"Deephaven, MN / Cedar Rapids, IA",106.425000,NaN,NaN
"Elkins Park, PA",211.500000,NaN,NaN
"Germantown, Philadelphia, PA",512.329200,NaN,NaN
"Haverford, PA",102.729150,NaN,NaN
"Haverford, PA / Cooperstown, NY",262.375000,NaN,NaN
"Lexington, MA",113.275000,NaN,NaN


# Aggregation methods

So far, we've only used `mean` as an aggregation method. But there are many others!

- `mean`
- `std`
- `sum`
- `count` (which works on non-numeric values, but doesn't include `NaN`)
- `median`
- `min`
- `max`
- `idxmin` and `idxmax` (returns the index of the min/max value)

In [44]:
# for every combination of party size + time, 
# what was the greatest tip recorded?

tips.pivot_table(index='size',
                 columns='time',
                 values='tip',
                 aggfunc='max', observed=True)

time,Lunch,Dinner
size,,
1,1.92,1.00
2,5.85,5.65
3,4.00,10.00
4,5.17,9.00
5,5.00,5.14
6,6.70,5.00


In [46]:

tips.pivot_table(index='size',
                 columns='time',
                 values='tip',
                 aggfunc='idxmax', observed=True)

time,Lunch,Dinner
size,,
1,222,67
2,88,181
3,200,170
4,85,212
5,142,155
6,141,156


In [49]:
tips.iloc[200]

total_bill    18.71
tip             4.0
sex            Male
smoker          Yes
day            Thur
time          Lunch
size              3
Name: 200, dtype: object

In [51]:
# what if we want to run more than one aggregation method?
# we can pass a list of aggregation functions to "aggfunc"!

df.pivot_table(index='sex',
               columns='pclass',
               values='fare',
               aggfunc=['count', 'min', 'max'])   # how many tickets? what were the lowest + highest fares paid

count             min                   max             
pclass   1.0  2.0  3.0   1.0   2.0   3.0       1.0   2.0    3.0
sex                                                            
female   144  106  216  25.7  10.5  6.75  512.3292  65.0  69.55
male     179  171  492   0.0   0.0  0.00  512.3292  73.5  69.55

In [52]:
# if I can pass a list of aggregation functions instead of one
# maybe I can pass a list of columns to be used for the index

df.pivot_table(index=['embarked', 'sex'],
               columns='pclass',
               values='fare',
               aggfunc='mean')

pclass                  1.0        2.0        3.0
embarked sex                                     
C        female  118.895949  27.003791  13.834545
         male     94.622560  20.904406   9.775901
Q        female   90.000000  12.350000   9.791968
         male     90.000000  11.489160  10.979167
S        female  101.069145  23.023118  18.083851
         male     53.670756  20.073322  13.145977

In [53]:

df.pivot_table(index=['embarked', 'sex'],
               columns='pclass',
               values='fare',
               aggfunc=['mean', 'std'])

mean                              std             \
pclass                  1.0        2.0        3.0        1.0        2.0   
embarked sex                                                              
C        female  118.895949  27.003791  13.834545  96.609289  12.253817   
         male     94.622560  20.904406   9.775901  96.214430   9.893143   
Q        female   90.000000  12.350000   9.791968   0.000000   0.000000   
         male     90.000000  11.489160  10.979167        NaN   1.232767   
S        female  101.069145  23.023118  18.083851  68.105876  11.115047   
         male     53.670756  20.073322  13.145977  49.900774  15.414203   

                            
pclass                 3.0  
embarked sex                
C        female   4.560882  
         male     4.482700  
Q        female   4.927602  
         male     6.958736  
S        female  14.015492  
         male    12.554548

In [55]:
df.pivot_table(index=['embarked', 'sex'],
               columns=['boat', 'pclass'],
               values='fare',
               aggfunc=['mean', 'std'])

mean                                             \
boat                    1          10                             11   
pclass                1.0         1.0        2.0       3.0       1.0   
embarked sex                                                           
C        female  48.26460         NaN  30.502100       NaN   42.7354   
         male    56.18545         NaN  37.004200   7.22920   57.7500   
Q        female       NaN         NaN  12.350000       NaN       NaN   
         male         NaN         NaN        NaN       NaN       NaN   
S        female       NaN  167.371862  23.861111  16.27605  103.7250   
         male    26.00000         NaN  22.750000  20.57500  151.5500   

                                                                ...       std  \
boat                                        12              13  ...         9   
pclass                 2.0      3.0        2.0    3.0      1.0  ...       3.0   
embarked sex                                                    ...             
C        female  13.791700      NaN  16.859350    NaN      NaN  ...       NaN   
         male          NaN      NaN        NaN    NaN      NaN  ...       NaN   
Q        female        NaN      NaN        NaN    NaN      NaN  ...       NaN   
         male          NaN      NaN        NaN    NaN      NaN  ...       NaN   
S        female  27.854167  14.1083  21.791667  16.10      NaN  ...       NaN   
         male    39.000000   9.4625  21.000000   8.05  81.8583  ...  0.072169   

                                                                               \
boat                    A                   B                    C              
pclass                1.0       3.0       1.0       3.0        1.0        3.0   
embarked sex                                                                    
C        female       NaN       NaN       NaN       NaN        NaN   5.097933   
         male    9.802268       NaN  58.25379       NaN        NaN   3.325729   
Q        female       NaN       NaN       NaN       NaN        NaN        NaN   
         male         NaN       NaN       NaN       NaN        NaN        NaN   
S        female       NaN  3.323402       NaN       NaN        NaN   0.466983   
         male         NaN  3.230564       NaN  0.377147  84.852814  21.738293   

                                           
boat                     D                 
pclass                 1.0  2.0       3.0  
embarked sex                               
C        female        NaN  NaN  0.000000  
         male          NaN  NaN       NaN  
Q        female        NaN  NaN  0.000000  
         male          NaN  NaN       NaN  
S        female  19.166699  NaN  0.106066  
         male    34.342042  0.0  5.868986  

[6 rows x 88 columns]

In [57]:
# what if I want to know the mean fare for all female passengers?
# or the mean fare for all people in 1st class?

# we can ask pivot_table to produce a new value at the edge of each row and column
# this is known as the "margin"

df.pivot_table(index='sex',
               columns='pclass',
               values='fare',
               aggfunc='mean',
              margins=True)

pclass,1.0,2.0,3.0,All
sex,,,,
female,109.412385,23.234827,15.324250,46.198097
male,69.888385,19.904946,12.415462,26.154601
All,87.508992,21.179196,13.302889,33.295479
